In [93]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()
import re

In [94]:
import requests
import urllib.request
import pandas as pd
import csv
from bs4 import BeautifulSoup
import pickle

# Web Scraping Data From Wikipedia
Most cities in the data set have very informative Wikipedia Sites. This data is publicaly avialable to anyone with a internet connection. As we're are particular interested in the comparing the CO2 emision with climate/geological data we're partilcuar interested in the section on Wikipedia page which describe a city's cliamte.  Below can a screenshot be seen of the Climate Data table for Berlin. 
![title](Berlin_climate.png)
Similar tables exisist for almost all the cities in the dataset. This data can be gatehered using the `BeautifulSoup` library. 

In [95]:
df = pd.read_csv(os.path.abspath('../data/Processed/Cities_imputed.csv'), index_col=0)

In [96]:
climate_df = df[['City', 'Country', 'Continent']]

In [97]:
climate_df['wiki_link'] = [re.sub(r'\([^)]*\)', '', city).replace(" ", "_") for city in climate_df.City]

In [98]:
climate_df.loc[climate_df['wiki_link'] == 'Birmingham']

,City,Country,Continent,wiki_link
319,Birmingham(AL),United States,North America,Birmingham
261,Birmingham,United Kingdom,Europe,Birmingham


## Correcting for the name of the city does land on the right Wikipage

In [99]:
def update_wiki_link(df, city, city_update):
    if city in df.wiki_link.values:
        df.loc[df['wiki_link'] == city, 'wiki_link'] = city_update
    else:
        print(city)

In [100]:
cities_updates = [
    ('Santa_Cruz', 'Santa_Cruz_(Bolivia)'),
    ('Cordoba' , 'Córdoba,_Argentina'),
    ('San_Francisco_Bay_Area', 'San_Fransisco'),
    ('Tampa-St._Petersburg', 'Saint_Petersburg_(Florida)'),
    ('Memphis', 'Memphis,_Tennessee'),
    ('Denver-Aurora', 'Aurora,_Colorado'),
    ('Rotterdam-Hague', 'Rotterdam'),
    ('Nashville-Davidson', 'Nashville,_Tennessee'),
    ('Cologne-Bonn', 'Cologne'),
    ('Richmond', 'Richmond,_Virginia'),
    ('Rochester', 'Rochester,_New_York'),
    ('Phoenix-Mesa', 'Phoenix_(Arizona)'),
    ('Leon', 'León,_Guanajuato'),
    ('Buffalo', 'Buffalo,_New_York'),
    ('Allentown-Bethlehem', 'Allentown,_Pennsylvania'),
    ('San_Juan', 'San_Juan,_Puerto_Rico'),
    ('Puebla', 'Puebla_(city)'),
    ('San_Jose', 'San_José,_Costa_Rica'),
    ('Dallas-Fort_Worth', 'Fort_Worth,_Texas'),
    ('Seoul-Incheon', 'Seoul'),
    ('Lome', 'Lomé'),
    ('Chihuahua', 'Chihuahua_City'),
    ('Fukuoka-Kitakyushu', 'Fukuoka'),
    ('Davao', 'Davao_City'),
    ('New_York', 'New_York_City'),
    ('Osaka-Kobe-Kyoto', 'Osaka'),
    ('Kansas_City', 'Kansas_City,_Missouri'),
    ('Minneapolis-St._Paul', 'Minneapolis'),
    ('Portland', 'Portland,_Oregon'),
    ('Phoenix_(Arizona)', 'Phoenix,_Arizona'),
    ('Washington', 'Washington,_D.C.'),
    ('Columbus', 'Columbus,_Ohio'),
    ('Salvador', 'Salvador,_Bahia'),
    ('Xi‚Äôan', 'Xi%27an'),
    ('Concepcion', 'Concepción,_Chile'),
    ('Providence', 'Providence,_Rhode_Island')
]

for city, city_update in cities_updates:
    update_wiki_link(
        df = climate_df,
        city = city,
        city_update = city_update
    )

In [101]:
climate_df.loc[(climate_df['wiki_link'] == 'Birmingham') & (climate_df['Country'] == 'United States')] = 'Birmingham,_Alabama'
climate_df.loc[(climate_df['wiki_link'] == 'Valencia') & (climate_df['Country'] == 'United Venezuela')] = 'Valencia,_Carabobo'

## Extracting the the climate data

In [102]:
def extract_wiki_page(wiki_url):
    response = requests.get(url)
    status_code = response.status_code

    soup = False
    if status_code == 200:
         soup = BeautifulSoup(response.text,"html.parser")

    return soup

In [103]:
def extract_climate_table(soup):
    # exstract the table with the cliamte data
    tables = soup.findAll('table',{"class":"collapsible"})

    select_table = None
    for table in tables:
        header = table.findAll(['th'])[0].getText()[0:12].lower()
        
        if 'climate' in header:
            select_table = table
            break
    
    return select_table

In [104]:
def process_climate_tabel(table):
    trs = table.findAll(['tr'])[1:]

    for i, cell in enumerate(trs):
        th = cell.find_all('th')
        th_data = [col.text.strip('\n') for col in th]  
        td = cell.find_all('td')
        row = [i.text.replace('\n','') for i in td]

        if i == 0:
            columns = th_data
            city_climate_df_tmp = pd.DataFrame(columns=columns)
        
        elif len(th_data) == 0:
            continue
        else:    
            tmp = pd.DataFrame({key: [value] for key, value in zip(columns, th_data+row)})
            city_climate_df_tmp = city_climate_df_tmp.append(tmp)
    
    city_climate_df_tmp.rename(columns={"Month": "Climate_variable"}, inplace = True)
    city_climate_df_tmp.set_index('Climate_variable', inplace = True)

    return city_climate_df_tmp

In [105]:
url = 'https://en.wikipedia.org/wiki/Toronto'
soup = extract_wiki_page(url)
city_climate_table = extract_climate_table(soup)
df_city_climate = process_climate_tabel(city_climate_table)
df_city_climate

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year
Climate_variable,,,,,,,,,,,,,
Record high humidex,15.7,12.2,21.7,31.6,39.8,44.5,43.0,42.6,43.8,31.2,26.1,17.7,44.5
Record high °C (°F),16.1(61.0),19.1(66.4),26.7(80.1),32.2(90.0),34.4(93.9),36.7(98.1),40.6(105.1),38.9(102.0),37.8(100.0),30.8(87.4),23.9(75.0),19.9(67.8),40.6(105.1)
Average high °C (°F),−0.7(30.7),0.4(32.7),4.7(40.5),11.5(52.7),18.4(65.1),23.8(74.8),26.6(79.9),25.5(77.9),21.0(69.8),14.0(57.2),7.5(45.5),2.1(35.8),12.9(55.2)
Daily mean °C (°F),−3.7(25.3),−2.6(27.3),1.4(34.5),7.9(46.2),14.1(57.4),19.4(66.9),22.3(72.1),21.5(70.7),17.2(63.0),10.7(51.3),4.9(40.8),−0.5(31.1),9.4(48.9)
Average low °C (°F),−6.7(19.9),−5.6(21.9),−1.9(28.6),4.1(39.4),9.9(49.8),14.9(58.8),18.0(64.4),17.4(63.3),13.4(56.1),7.4(45.3),2.3(36.1),−3.1(26.4),5.9(42.6)
Record low °C (°F),−32.8(−27.0),−31.7(−25.1),−26.7(−16.1),−15.0(5.0),−3.9(25.0),−2.2(28.0),3.9(39.0),4.4(39.9),−2.2(28.0),−8.9(16.0),−20.6(−5.1),−30.0(−22.0),−32.8(−27.0)
Record low wind chill,−37,−34,−26,−17,−8,0,0,0,0,−8,−17,−34,−37
Average precipitation mm (inches),61.5(2.42),55.4(2.18),53.7(2.11),68.0(2.68),82.0(3.23),70.9(2.79),63.9(2.52),81.1(3.19),84.7(3.33),64.4(2.54),84.1(3.31),61.5(2.42),831.1(32.72)
Average rainfall mm (inches),29.1(1.15),29.7(1.17),33.6(1.32),61.1(2.41),82.0(3.23),70.9(2.79),63.9(2.52),81.1(3.19),84.7(3.33),64.3(2.53),75.4(2.97),38.2(1.50),714.0(28.11)


### Excrating all cities climate data

In [106]:
climate_data_dict = dict()

for wiki_link in climate_df.wiki_link:
    url = 'https://en.wikipedia.org/wiki/' + wiki_link
    soup = extract_wiki_page(url)

    if soup:
        city_climate_table = extract_climate_table(soup)

        # If the climate table is available on Wikipedia
        if city_climate_table:
            df_city_climate = process_climate_tabel(city_climate_table)
            climate_data_dict[wiki_link] = df_city_climate

        # If the wikipedia page does not have climate table
        else: 
            climate_data_dict[wiki_link] = None
            print('No climate data available for city: ', wiki_link)
    else:
        print('Could not find citys: ', wiki_link, 'Wikipedia page')

No climate data available for city:  Cebu
No climate data available for city:  Caracas
No climate data available for city:  Santo_Domingo
No climate data available for city:  Monterrey


In [107]:
# Save the climate data in pickle as it take quite some time to geneate.
with open('climate_data_dict.pickle', 'wb') as handle:
    pickle.dump(climate_data_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Processing the climate data

In [121]:
with open('climate_data_dict.pickle', 'rb') as handle:
    climate_data_dict = pickle.load(handle)

In [109]:
climate_data_dict['Austin']

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year
Climate_variable,,,,,,,,,,,,,
Record high °F (°C),90(32),99(37),98(37),99(37),104(40),109(43),109(43),112(44),112(44),100(38),91(33),90(32),112(44)
Mean maximum °F (°C),79.3(26.3),83.3(28.5),86.6(30.3),91.7(33.2),95.4(35.2),98.1(36.7),101.0(38.3),102.9(39.4),99.1(37.3),93.3(34.1),84.8(29.3),79.7(26.5),103.9(39.9)
Average high °F (°C),61.5(16.4),65.2(18.4),72.2(22.3),79.8(26.6),86.5(30.3),92.1(33.4),95.6(35.3),97.0(36.1),90.5(32.5),81.8(27.7),71.4(21.9),62.7(17.1),79.8(26.6)
Average low °F (°C),41.5(5.3),44.8(7.1),51.3(10.7),58.6(14.8),66.7(19.3),72.3(22.4),74.4(23.6),74.6(23.7),69.4(20.8),60.6(15.9),50.6(10.3),42.3(5.7),59.0(15.0)
Mean minimum °F (°C),26.7(−2.9),28.5(−1.9),34.7(1.5),42.9(6.1),54.1(12.3),64.2(17.9),69.9(21.1),69.0(20.6),56.9(13.8),44.7(7.1),34.6(1.4),26.7(−2.9),22.0(−5.6)
Record low °F (°C),−2(−19),−1(−18),18(−8),30(−1),40(4),51(11),57(14),58(14),41(5),30(−1),20(−7),4(−16),−2(−19)
Average precipitation inches (mm),2.22(56),2.02(51),2.76(70),2.09(53),4.44(113),4.33(110),1.88(48),2.35(60),2.99(76),3.88(99),2.96(75),2.40(61),34.32(872)
Average snowfall inches (cm),0.4(1.0),0.2(0.51),0(0),0(0),0(0),0(0),0(0),0(0),0(0),0(0),0(0),trace,0.6(1.5)
Average precipitation days (≥ 0.01 in),7.4,7.4,9.2,7.1,8.9,7.7,5.4,4.9,6.7,7.5,7.5,7.8,87.5


In [110]:
climate_data_dict['Freetown']

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year
Climate_variable,,,,,,,,,,,,,
Record high °C (°F),36.0(96.8),38.0(100.4),38.0(100.4),38.5(101.3),35.0(95.0),35.0(95.0),33.0(91.4),32.0(89.6),31.0(87.8),35.0(95.0),36.5(97.7),33.5(92.3),38.5(101.3)
Average high °C (°F),29.9(85.8),30.3(86.5),30.9(87.6),31.2(88.2),30.9(87.6),30.1(86.2),28.7(83.7),28.4(83.1),29.0(84.2),29.9(85.8),30.1(86.2),29.7(85.5),29.9(85.9)
Daily mean °C (°F),27.3(81.1),27.6(81.7),28.1(82.6),28.5(83.3),28.3(82.9),27.4(81.3),26.3(79.3),25.9(78.6),26.4(79.5),27.1(80.8),27.7(81.9),27.5(81.5),27.3(81.2)
Average low °C (°F),23.8(74.8),24.0(75.2),24.4(75.9),24.8(76.6),24.4(75.9),23.6(74.5),23.1(73.6),23.0(73.4),23.1(73.6),23.4(74.1),24.0(75.2),24.1(75.4),23.8(74.9)
Record low °C (°F),15.0(59.0),17.8(64.0),19.4(66.9),20.0(68.0),20.6(69.1),20.0(68.0),19.4(66.9),19.4(66.9),20.0(68.0),19.4(66.9),20.0(68.0),15.6(60.1),15.0(59.0)
Average rainfall mm (inches),8.0(0.31),6.0(0.24),28.0(1.10),68.0(2.68),214.0(8.43),522.0(20.55),"1,190(46.85)","1,078(42.44)",800.0(31.50),333.0(13.11),148.0(5.83),38.0(1.50),"4,433(174.54)"
Average rainy days,0,0,1,4,15,22,27,27,24,21,9,2,152
Average relative humidity (%) (at 15:00 LST),66,67,67,68,73,76,81,82,80,78,76,69,74
Mean monthly sunshine hours,226.3,215.6,232.5,207.0,189.1,153.0,102.3,86.8,126.0,186.0,198.0,161.2,"2,083.8"


In [111]:
city_climate_list = [list(city_climate_df.index.values) for city, city_climate_df in climate_data_dict.items() if city_climate_df is not None]
climate_variables = [city for sublist in city_climate_list for city in sublist]
set_of_climate_variable = set(climate_variables)

In [112]:
pd.Series(climate_variables).value_counts()

Mean monthly sunshine hours                      298
Average high °C (°F)                             270
Average low °C (°F)                              270
Record low °C (°F)                               254
Record high °C (°F)                              254
                                                ... 
Average relative humidity (%) (daily average)      1
Average relative humidity (%) (at 15:00 UTC)       1
Average rainy days (≥ 0.01 mm)                     1
Average snowy days (≥ 1 cm)                        1
Average relative humidity (%) (at 14:00)           1
Length: 69, dtype: int64

In [113]:
set_of_climate_variable

{'Average afternoon relative humidity (%)',
 'Average dew point °C (°F)',
 'Average dew point °F (°C)',
 'Average high °C (°F)',
 'Average high °F (°C)',
 'Average low °C (°F)',
 'Average low °F (°C)',
 'Average precipitation days',
 'Average precipitation days (≥ 0.005 inch)',
 'Average precipitation days (≥ 0.01 in)',
 'Average precipitation days (≥ 0.01 in.)',
 'Average precipitation days (≥ 0.01 inch)',
 'Average precipitation days (≥ 0.01\xa0mm)',
 'Average precipitation days (≥ 0.1 mm)',
 'Average precipitation days (≥ 0.1\xa0mm)',
 'Average precipitation days (≥ 0.2 mm)',
 'Average precipitation days (≥ 0.2\xa0mm)',
 'Average precipitation days (≥ 0.5 mm)',
 'Average precipitation days (≥ 1.0 mm)',
 'Average precipitation days (≥ 1\xa0mm)',
 'Average precipitation inches (mm)',
 'Average precipitation mm (inches)',
 'Average rainfall inches (mm)',
 'Average rainfall mm (inches)',
 'Average rainy days',
 'Average rainy days (≥ 0.01 in)',
 'Average rainy days (≥ 0.01 inch)',
 'Ave

In [114]:
rename_column = {
    'Mean monthly sunshine hours' : 'Mean monthly sunshine hours',
    'Average high °C (°F)'  : 'Average high °C',
    'Average high °F (°C)'  : 'Average high °C',
    'Average low °C (°F)'   : 'Average low °C',
    'Average low °F (°C)'   : 'Average low °C',
    'Average rainfall inches (mm)'      : 'Average precipitation mm',
    'Average rainfall mm (inches)'      : 'Average precipitation mm',
    'Average precipitation inches (mm)' : 'Average precipitation mm',
    'Average precipitation mm (inches)' : 'Average precipitation mm',
    'Average precipitation mm'          : 'Average precipitation mm',
    'Average precipitation days'                : 'Average precipitation days',
    'Average precipitation days (≥ 0.005 inch)' : 'Average precipitation days',
    'Average precipitation days (≥ 0.01 in)'    : 'Average precipitation days',
    'Average precipitation days (≥ 0.01 in.)'   : 'Average precipitation days',
    'Average precipitation days (≥ 0.01 inch)'  : 'Average precipitation days',
    'Average precipitation days (≥ 0.01\xa0mm)' : 'Average precipitation days',
    'Average precipitation days (≥ 0.1 mm)'     : 'Average precipitation days',
    'Average precipitation days (≥ 0.1\xa0mm)'  : 'Average precipitation days',
    'Average precipitation days (≥ 0.2 mm)'     : 'Average precipitation days',
    'Average precipitation days (≥ 0.2\xa0mm)'  : 'Average precipitation days',
    'Average precipitation days (≥ 0.5 mm)'     : 'Average precipitation days',
    'Average precipitation days (≥ 1.0 mm)'     : 'Average precipitation days',
    'Average precipitation days (≥ 1\xa0mm)'    : 'Average precipitation days',
    'Average rainy days'                        : 'Average precipitation days',
    'Average rainy days (≥ 0.01 in)'            : 'Average precipitation days',
    'Average rainy days (≥ 0.01 inch)'          : 'Average precipitation days',
    'Average rainy days (≥ 0.01 mm)'            : 'Average precipitation days',
    'Average rainy days (≥ 0.1 mm)'             : 'Average precipitation days',
    'Average rainy days (≥ 0.2 mm)'             : 'Average precipitation days',
    'Average rainy days (≥ 0.5 mm)'             : 'Average precipitation days',
    'Average rainy days (≥ 1 mm)'               : 'Average precipitation days',
    'Average rainy days (≥ 1.0 mm)'             : 'Average precipitation days',
    'Average rainy days (≥ 1.0\xa0mm)'          : 'Average precipitation days',
    'Average rainy days (≥ 1mm)'                : 'Average precipitation days',
}

In [115]:
def select_celcius(row):

    matric = []
    for imperial_metric in row:
        try:
            val1, val2 = imperial_metric.replace(")", "").split('(')
            
            # Remove symbol which look like a minus sign which is not not a minus sign
            val1, val2 = val1.replace('−', '-'), val2.replace('−', '-')
            # Remove commas which seperates tree zeros and convert to float 
            val1, val2 = float(val1.replace(',', '')), float(val2.replace(',', ''))

            matric.append(min(val1, val2))
        except ValueError:
            try:
                matric.append(float(imperial_metric))
            except ValueError:
                matric.append(None)
    return matric

In [116]:
def select_mm(row):

    matric = []
    for imperial_metric in row:
        try:
            val1, val2 = imperial_metric.replace(")", "").split('(')
            
            # Remove symbol which look like a minus sign which is not not a minus sign
            val1, val2 = val1.replace('−', '-'), val2.replace('−', '-')
            # Remove commas which seperates tree zeros and convert to float 
            val1, val2 = float(val1.replace(',', '')), float(val2.replace(',', ''))

            matric.append(max(val1, val2))
        except ValueError:
            try:
                matric.append(float(imperial_metric))
            except ValueError:
                matric.append(None)
    return matric

In [117]:
def keep_selected_varibles(city_climate_df, rename_column):
    
    # Rename remaing climate variables:
    city_climate_variables = city_climate_df.index.tolist()
    name_change  = {climate_variable : rename_column[climate_variable] for climate_variable  in city_climate_variables if climate_variable in rename_column}
    city_climate_df.rename(name_change, inplace = True)

    # only keep the variables where the name was updated:
    city_climate_df_update = city_climate_df.loc[name_change.values()]

    # Remove duplicate indexes...
    # This can happen if a city e.g. Toronto has 'Average precipitation mm (inches)',
    # 'Average rainfall mm (inches)', 'Average snowfall cm (inches)' i.e. precipitation = rainfall + snowfall
    # THis method keeps the first index. Hopefully in all cases this it the total precipitation.
    city_climate_df_update = city_climate_df_update[~city_climate_df_update.index.duplicated(keep='first')]

    return city_climate_df_update


def convert_to_metric(city_climate_df, function_dict):
    
    columns = city_climate_df.columns
    metric_city_climate_df = pd.DataFrame(columns = columns)

    climate_variables = []
    for climate_var, row in city_climate_df.iterrows():
        climate_variables.append(climate_var)
        metric = function_dict[climate_var]( city_climate_df.loc[climate_var] )

        tmp = pd.DataFrame({key: [value] for key, value in zip(columns, metric)})
        metric_city_climate_df = metric_city_climate_df.append(tmp)

    metric_city_climate_df['Climate_variable'] = climate_variables
    metric_city_climate_df.set_index('Climate_variable', inplace = True)

    return metric_city_climate_df

In [122]:
city_climate_df = climate_data_dict['Baltimore']

In [123]:
# Keep only selected columns and remname them:
city_climate_df_update = keep_selected_varibles(
    city_climate_df = city_climate_df,
    rename_column = rename_column
)


In [135]:
row = city_climate_df_update.loc['Mean monthly sunshine hours']

In [139]:
def select_sunshine(row):
    matric = []
    for val in row:
        try:
            val = float(val.replace(',', ''))
            matric.append(val)
        except ValueError:
            matric.append(np.nan)
    return metric

In [140]:
matric

[155.4,
 164.0,
 215.0,
 230.7,
 254.5,
 277.3,
 290.1,
 264.4,
 221.8,
 205.5,
 158.5,
 144.5,
 2581.7]

In [134]:
tmp.values

array(['155.4', '164.0', '215.0', '230.7', '254.5', '277.3', '290.1',
       '264.4', '221.8', '205.5', '158.5', '144.5', '2,581.7'],
      dtype=object)

In [125]:

# extract the metric representaiton
metric_city_climate_df = convert_to_metric(
    city_climate_df = city_climate_df_update,
    function_dict = function_dict
)

TypeError: cannot convert the series to <class 'float'>

In [118]:
function_dict = {
    'Average high °C' : select_celcius,
    'Average low °C' : select_celcius,
    'Average precipitation mm' : select_mm,
    'Average precipitation days' : lambda x : x,
    'Mean monthly sunshine hours' : lambda x : float(x.replace(",", ""))
}

index = pd.Index(['Average high °C (°F)', 'Average low °C (°F)','Average precipitation mm', 'Average precipitation days'], dtype='object', name='Climate_variable')

empty_df = pd.DataFrame({
    'Jan'   : np.repeat(np.nan, 4),
    'Feb'   : np.repeat(np.nan, 4),
    'Mar'   : np.repeat(np.nan, 4),
    'Apr'   : np.repeat(np.nan, 4),
    'May'   : np.repeat(np.nan, 4), 
    'Jun'   : np.repeat(np.nan, 4),
    'Jul'   : np.repeat(np.nan, 4),
    'Aug'   : np.repeat(np.nan, 4),
    'Sep'   : np.repeat(np.nan, 4),
    'Oct'   : np.repeat(np.nan, 4),
    'Nov'   : np.repeat(np.nan, 4),
    'Dec'   : np.repeat(np.nan, 4),
    'Year'  : np.repeat(np.nan, 4),
}, index = index)

cleaned_city_dict = dict()
for i, (city, city_climate_df) in enumerate(climate_data_dict.items()):

    if city_climate_df is None:
        cleaned_city_dict[city] = empty_df
        continue

    # Keep only selected columns and remname them:
    city_climate_df_update = keep_selected_varibles(
        city_climate_df = city_climate_df,
        rename_column = rename_column
    )

    # extract the metric representaiton
    metric_city_climate_df = convert_to_metric(
        city_climate_df = city_climate_df_update,
        function_dict = function_dict
    )

    # Add missing vlaues
    metric_city_climate_df = metric_city_climate_df.append(empty_df)
    metric_city_climate_df = metric_city_climate_df[~metric_city_climate_df.index.duplicated(keep='first')]
    metric_city_climate_df = metric_city_climate_df.reindex(index)


    # Update the city dataframe such that it only keeps the choosen climate variables
    # and they are metric
    cleaned_city_dict[city] = metric_city_climate_df

TypeError: cannot convert the series to <class 'float'>

In [120]:
city_climate_df

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year
Climate_variable,,,,,,,,,,,,,
Average high °C,42.4(5.8),45.7(7.6),53.9(12.2),65.5(18.6),75.2(24.0),85.0(29.4),89.0(31.7),87.0(30.6),80.3(26.8),68.4(20.2),57.7(14.3),46.1(7.8),66.4(19.1)
Average low °C,29.2(−1.6),31.4(−0.3),38.8(3.8),47.6(8.7),56.9(13.8),67.1(19.5),72.5(22.5),70.9(21.6),63.8(17.7),51.8(11.0),42.9(6.1),32.8(0.4),50.5(10.3)
Average precipitation mm,2.92(74),2.60(66),3.86(98),3.22(82),3.49(89),3.27(83),4.62(117),3.39(86),4.09(104),3.05(77),2.97(75),3.41(87),"40.89(1,038)"
Average snowfall inches (cm),6.8(17),8.0(20),1.9(4.8),trace,0(0),0(0),0(0),0(0),0(0),0(0),0.4(1.0),3.0(7.6),20.1(51)
Average precipitation days,9.5,8.4,10.5,11.1,11.2,10.8,10.7,9.2,8.9,8.3,8.8,9.9,117.3
Average snowy days (≥ 0.1 in),3.5,2.8,1.1,0.1,0,0,0,0,0,0,0.3,1.7,9.5
Average relative humidity (%),63.2,61.3,59.2,58.9,66.1,68.4,69.1,71.1,71.3,69.5,66.5,65.5,65.8
Mean monthly sunshine hours,155.4,164.0,215.0,230.7,254.5,277.3,290.1,264.4,221.8,205.5,158.5,144.5,"2,581.7"
Percent possible sunshine,51,54,58,58,57,62,64,62,59,59,52,49,58


In [27]:
cleaned_city_dict['Austin']

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year
Climate_variable,,,,,,,,,,,,,
Average high °C (°F),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Average low °C (°F),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Average precipitation mm,56,51,70,53,113,110,48,60,76,99,75,61,872
Average precipitation days,7.4,7.4,9.2,7.1,8.9,7.7,5.4,4.9,6.7,7.5,7.5,7.8,87.5


In [151]:
cleaned_city_dict['Freetown']

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year
Climate_variable,,,,,,,,,,,,,
Average high °C (°F),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Average low °C (°F),NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Average precipitation mm,8,6,28,68,214,522,1190,1078,800,333,148,38,4433
Average precipitation days,0,0,1,4,15,22,27,27,24,21,9,2,152


## Add the data

In [112]:
def stack_climate_data(city_climate_df):
    stacked_city_climate_date = city_climate_df.stack().to_frame()
    stacked_city_climate_date.index = stacked_city_climate_date.index.map('_'.join)

    return stacked_city_climate_date.T

In [109]:
stacked_city_climate_date = cleaned_city_dict['Freetown'].stack().to_frame()
stacked_city_climate_date.index = stacked_city_climate_date.index.map('_'.join)

In [115]:
climate_df

,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec,Year
Climate_variable,,,,,,,,,,,,,
Record high °C (°F),39.3(102.7),38.3(100.9),43.0(109.4),38.6(101.5),33.5(92.3),29.8(85.6),29.0(84.2),32.0(89.6),33.1(91.6),37.2(99.0),39.9(103.8),41.4(106.5),43.0(109.4)
Mean maximum °C (°F),33.6(92.5),34.1(93.4),33.2(91.8),31.7(89.1),29.1(84.4),26.3(79.3),25.1(77.2),26.9(80.4),28.3(82.9),31.0(87.8),31.6(88.9),32.5(90.5),34.1(93.4)
Average high °C (°F),26.1(79.0),26.5(79.7),25.4(77.7),23.0(73.4),20.3(68.5),18.1(64.6),17.5(63.5),17.8(64.0),19.2(66.6),21.3(70.3),23.5(74.3),24.9(76.8),22.0(71.5)
Daily mean °C (°F),20.4(68.7),20.4(68.7),19.2(66.6),16.9(62.4),14.4(57.9),12.5(54.5),11.9(53.4),12.4(54.3),13.7(56.7),15.6(60.1),17.9(64.2),19.5(67.1),16.2(61.2)
Average low °C (°F),15.7(60.3),15.6(60.1),14.2(57.6),11.9(53.4),9.4(48.9),7.8(46.0),7.0(44.6),7.5(45.5),8.7(47.7),10.6(51.1),13.2(55.8),14.9(58.8),11.4(52.5)
Mean minimum °C (°F),10.3(50.5),9.9(49.8),7.6(45.7),5.7(42.3),2.8(37.0),1.3(34.3),1.0(33.8),1.3(34.3),2.3(36.1),4.4(39.9),7.0(44.6),9.5(49.1),1.0(33.8)
Record low °C (°F),7.4(45.3),6.4(43.5),4.6(40.3),2.4(36.3),0.9(33.6),−1.2(29.8),−1.3(29.7),−0.4(31.3),0.2(32.4),1.0(33.8),3.9(39.0),6.2(43.2),−1.3(29.7)
Average precipitation mm,15(0.6),17(0.7),20(0.8),41(1.6),69(2.7),93(3.7),82(3.2),77(3.0),40(1.6),30(1.2),14(0.6),17(0.7),515(20.4)
Average precipitation days,5.5,4.6,4.8,8.3,11.4,13.3,11.8,13.7,10.4,8.7,4.9,6.3,103.7


In [113]:
stack_climate_data(cleaned_city_dict['Baltimore'])

,Average high °C (°F)_Jan,Average high °C (°F)_Feb,Average high °C (°F)_Mar,Average high °C (°F)_Apr,Average high °C (°F)_May,Average high °C (°F)_Jun,Average high °C (°F)_Jul,Average high °C (°F)_Aug,Average high °C (°F)_Sep,Average high °C (°F)_Oct,...,Average precipitation days_Apr,Average precipitation days_May,Average precipitation days_Jun,Average precipitation days_Jul,Average precipitation days_Aug,Average precipitation days_Sep,Average precipitation days_Oct,Average precipitation days_Nov,Average precipitation days_Dec,Average precipitation days_Year
0,5.8,7.6,12.2,18.6,24,29.4,31.7,30.6,26.8,20.2,...,11.1,11.2,10.8,10.7,9.2,8.9,8.3,8.8,9.9,117.3


In [87]:
[(city, df.shape) for city, df in cleaned_city_dict.items()]

[('Baltimore', (4, 13)),
 ('Melbourne', (4, 13)),
 ('Niamey', (4, 13)),
 ('Hanoi', (4, 13)),
 ('Urumqi', (4, 13)),
 ('Milwaukee', (4, 13)),
 ('Paris', (4, 13)),
 ('Montevideo', (4, 13)),
 ('Krakow', (4, 13)),
 ('Yekaterinburg', (4, 13)),
 ('Pune', (4, 13)),
 ('Cebu', (4, 13)),
 ('Abidjan', (4, 13)),
 ('Austin', (4, 13)),
 ('Yaounde', (4, 13)),
 ('Delhi', (4, 13)),
 ('Hefei', (4, 13)),
 ('Dubai', (4, 13)),
 ('Chicago', (4, 13)),
 ('Dalian', (4, 13)),
 ('Ho_Chi_Minh_City', (4, 13)),
 ('Kharkiv', (4, 13)),
 ('Managua', (4, 13)),
 ('Kathmandu', (4, 13)),
 ('Chittagong', (4, 13)),
 ('Karachi', (4, 13)),
 ('Glasgow', (4, 13)),
 ('Huambo', (4, 13)),
 ('Santa_Cruz_(Bolivia)', (4, 13)),
 ('Lille', (4, 13)),
 ('Baghdad', (4, 13)),
 ('Rabat', (4, 13)),
 ('Chongqing', (4, 13)),
 ('Cologne', (4, 13)),
 ('Algiers', (4, 13)),
 ('Bangalore', (4, 13)),
 ('Mombasa', (4, 13)),
 ('Barcelona', (4, 13)),
 ('Ulsan', (4, 13)),
 ('Johor_Bahru', (4, 13)),
 ('Atlanta', (4, 13)),
 ('Samara', (4, 13)),
 ('Toronto'